In [1]:
%load_ext autoreload
%autoreload 2


from util import *
from pastalgorithm import algorithm_cost,algorithm_GA
from myalgorithm import algorithm_basic_greedy,algorithm_regret,algorithm_basic_greedy_v2
from util_zoe.myalgorithm import algorithm

## 개별 문제 실행

In [149]:

problem_file = './stage1_problem/STAGE1_4.json'
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

alg_start_time = time.time()

exception = None

solution = None
solution = algorithm_basic_greedy_v2(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

Best obj = 7428.685
0 th current_solution 7377.184
10 th current_solution 6896.755999999999
20 th current_solution 6445.581499999999
30 th current_solution 6020.453499999998
40 th current_solution 5611.835499999998
50 th current_solution 5222.1895
60 th current_solution 4848.4255
70 th current_solution 4490.487000000001
80 th current_solution 4154.1105
90 th current_solution 3834.585500000001
100 th current_solution 3535.6684999999993
110 th current_solution 3258.8044999999997
120 th current_solution 3006.6265000000008
130 th current_solution 2815.7205000000013
performance before local search : 2763.6275
local search cost: 2741.4140000000007
[184]
[146]
[185]
[186]
[127]
[184]
[146]
local search cost: 2736.9889999999996
[184]
[146]
[184]
[146]
local search cost: 2736.7705
[184]
[146]
local search cost: 2735.4639999999995
local search cost: 2734.778
local search cost: 2734.7779999999993
break
best


In [ ]:

problem_file = './stage1_problem/STAGE1_14.json'
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

alg_start_time = time.time()

exception = None

solution = None
try:
    # Run algorithm!
    solution = algorithm_basic_greedy_v2(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)
except Exception as e:
    exception = f'{e}'


alg_end_time = time.time()

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

checked_solution['time'] = alg_end_time - alg_start_time
checked_solution['timelimit_exception'] = (alg_end_time - alg_start_time) > timelimit + 1 # allowing additional 1 second!
checked_solution['exception'] = exception

checked_solution['prob_name'] = prob['name']
checked_solution['prob_file'] = problem_file

checked_solution

In [ ]:
draw_route_solution(ALL_ORDERS, checked_solution)

In [ ]:
draw_bundle_solution(ALL_ORDERS, ALL_RIDERS, DIST, checked_solution)

## 전체 문제 실행

In [80]:
avg_cost = []

In [150]:
# baseline='avg_cost': 4903.952,
# 'avg_cost': 4118.8099999999995,
# problem_file = 'STAGE1_1.json'
# avg_cost = []
for i in range(14,19):

    problem_file = f'./stage1_problem/STAGE1_{i}.json'
    timelimit = 60

    # np.random.seed(1)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST']) # 100*100 주문,도착지 pair가 order 50개에 들어있음.
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time) # T라는 속성에 커스텀 거리 행렬 들어감.

    alg_start_time = time.time()

    exception = None

    solution = None
  
    solution = algorithm_basic_greedy_v2(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)
    

    alg_end_time = time.time()
    print(f'total elapsed time : {alg_end_time-alg_start_time}')
    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)
    avg_cost.append((K,checked_solution['avg_cost'],alg_end_time-alg_start_time))

Best obj = 7983.7
0 th current_solution 7893.08
10 th current_solution 7111.5109999999995
20 th current_solution 6409.607
30 th current_solution 5794.423000000001
40 th current_solution 5219.272999999999
50 th current_solution 4715.82
60 th current_solution 4339.622
performance before local search : 4339.622
local search cost: 4328.852000000001
local search cost: 4328.852
local search cost: 4324.951999999999
local search cost: 4320.959999999999
local search cost: 4319.8679999999995
local search cost: 4319.508
local search cost: 4318.2
local search cost: 4313.331999999999
local search cost: 4313.143999999999
local search cost: 4293.3859999999995
local search cost: 4283.6320000000005
local search cost: 4277.996
local search cost: 4265.468000000001
local search cost: 4265.468
break
best
total elapsed time : 59.92647981643677
Best obj = 8064.64
0 th current_solution 8005.946
10 th current_solution 7486.455
20 th current_solution 7051.931
30 th current_solution 6637.627
40 th current_soluti

## 주문 수 별 평균 계산

In [152]:

# 100일 때와 200일 때의 avg_cost 합계와 개수를 계산
sum_100 = sum_200 = count_100 = count_200 = count_300 = sum_300 = 0

for k, cost,_ in avg_cost:
    if k == 100:
        sum_100 += cost
        count_100 += 1
    elif k == 200:
        sum_200 += cost
        count_200 += 1
    elif k == 300:
        sum_300 += cost
        count_300 += 1


# 평균 계산
avg_100 = sum_100 / count_100 if count_100 != 0 else 0
avg_200 = sum_200 / count_200 if count_200 != 0 else 0

avg_300 = sum_300 / count_300 if count_300 != 0 else 0

# 결과를 리스트에 추가
avg_cost.append(('mean',100, avg_100))
avg_cost.append(('mean',200, avg_200))

avg_cost.append(('mean',300, avg_300))

avg_cost

[(100, 3683.885999999999, 59.94561767578125),
 (100, 3308.2659999999987, 59.90532898902893),
 (200, 3271.1775, 60.03303074836731),
 (200, 2753.5044999999996, 59.92299842834473),
 (300, 2550.227, 60.01406717300415),
 (300, 2347.529, 59.95720672607422),
 (100, 3408.207999999999, 59.91824412345886),
 (100, 3331.6819999999993, 59.909656286239624),
 (200, 3306.8344999999995, 60.02853298187256),
 (200, 2721.068999999999, 59.93531775474548),
 (300, 2809.093999999999, 59.94492149353027),
 (300, 2425.409333333333, 59.94384956359863),
 (100, 4825.861999999999, 59.92682194709778),
 (100, 4265.468, 59.92647981643677),
 (200, 4466.805500000001, 60.02234172821045),
 (200, 3326.3884999999987, 59.91646909713745),
 (300, 3769.9580000000005, 60.11322283744812),
 (300, 3244.9813333333327, 59.98920822143555),
 ('mean', 100, 3803.8953333333325),
 ('mean', 200, 3307.6299166666663),
 ('mean', 300, 2857.866444444444)]

In [153]:
with open('algorithm_basic_greedy_local_search.txt', 'w') as file:
    for number in avg_cost:
        file.write(f"{number}\n")

## 문제 상황 그래프 

In [ ]:
for i in range(10,18):
    
    problem_file = f'./stage1_problem/STAGE1_{i+1}.json'
    
    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']
    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    plt.subplots(figsize=(8, 8))
    node_size = 5

    shop_x = [order.shop_lon for order in ALL_ORDERS]
    shop_y = [order.shop_lat for order in ALL_ORDERS]
    plt.scatter(shop_x, shop_y, c='red', s=node_size, label='SHOPS')

    dlv_x = [order.dlv_lon for order in ALL_ORDERS]
    dlv_y = [order.dlv_lat for order in ALL_ORDERS]
    plt.scatter(dlv_x, dlv_y, c='blue', s=node_size, label='DLVS')
    # 매장과 배송지를 잇는 선을 그립니다.
    for sx, sy, dx, dy in zip(shop_x, shop_y, dlv_x, dlv_y):
        plt.plot([sx, dx], [sy, dy], 'gray', linestyle='-', linewidth=0.5)  # 회색 선으로 매장과 배송지를 연결
